In [61]:
import lightgbm 
import os
from google.colab import drive
import pandas as pd
import numpy as np
import pickle

import requests

In [62]:
#Levanto drive para poder almacenar los archivos
drive.mount("/content/drive")
contentFolder = "/content/drive/MyDrive/Austral - Data Mining"
if not os.path.exists(contentFolder):
    os.mkdir(contentFolder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
def preprocesamiento(data):
  for x in [y for y in data.columns if "fecha" in y]:
      data[x] = pd.to_datetime(data[x], format = "%Y-%m-%d")


  medianasProvincia = data[data.edad.isnull()==False].groupby("residencia_provincia_nombre").edad.median()\
  .reset_index().rename(columns={"edad":"medianaEdad"})
  data = pd.merge(data, medianasProvincia, how="left", on="residencia_provincia_nombre")
  #Imputo la variable edad (aquellos nulos)
  data["edad"] = np.where(data.edad.isnull(), data.medianaEdad, data.edad)
  data["edad"] = np.where(data.edad <0 , data.edad * -1,data.edad)
  data["edad"] = np.where(data.edad > 110, 110, data.edad)
  #Drop de medianaEdad
  data = data.drop(columns=["medianaEdad"])
  data["decileEdad"] = pd.qcut(data.edad, 10, labels=False)
  data["edad"] = np.where(data.edad_años_meses == "Meses", round(data.edad/12,2), data.edad)

  #Creo variable dummies con sexo
  data = data.join(pd.get_dummies(data.sexo, prefix="sexo"))
  #Creo variable dummy con financiamiento
  data = data.join(pd.get_dummies(data.origen_financiamiento, prefix="financiamiento"))

  data["confirmado"] = np.where(data.clasificacion_resumen == "Confirmado", 1, 0)
  data["descartado"] = np.where(data.clasificacion_resumen == "Descartado", 1, 0)
  data["otros"] = np.where(~data.clasificacion_resumen.isin(["Confirmado", "Descartado"]), 1, 0)

  aux = data.groupby("carga_provincia_nombre").agg({
    "confirmado": "sum",
    "descartado": "sum"
  }).reset_index()
  aux["ratioDescartadoConfirmadoProvincia"] = aux["descartado"] / aux["confirmado"]
  data = pd.merge(data, aux[["carga_provincia_nombre", "ratioDescartadoConfirmadoProvincia"]], how="inner", on="carga_provincia_nombre")

  for x in ["cuidado_intensivo", "fallecido", "asistencia_respiratoria_mecanica"]:
    data[x] = np.where(data[x] == "SI",1,0)

  borrarColumnas = ["edad_años_meses", "residencia_provincia_id", "residencia_departamento_id",
                  "ultima_actualizacion","carga_provincia_id"]
  data.drop(columns = borrarColumnas, inplace=True)

  medianaGenero = data.groupby("sexo").agg({
    "edad":["median", "max"]
  }).reset_index()
  medianaGenero.columns = ["sexo", "medianaEdad", "maxEdad"]
  #Creamos sólo el ratio contra la mediana porque no nos sirve el maximo de edad
  data = pd.merge(data, medianaGenero[["sexo", "medianaEdad"]], how="inner", on = "sexo")
  data["ratioEdadGenero"] = data["edad"] / data["medianaEdad"]

  data["residencia_pais_nombre"] = np.where(data.residencia_pais_nombre == "SIN ESPECIFICAR", "Argentina", data.residencia_pais_nombre)
  data["extranjero"] = np.where(data.residencia_pais_nombre != "Argentina",1,0)

  medianaProvincia = data.groupby("residencia_provincia_nombre").agg({
    "edad":["median", "max"]
  }).reset_index()
  medianaProvincia.columns = ["residencia_provincia_nombre", "medianaEdadProvincia", "maxEdad"]

  data = pd.merge(data, medianaProvincia[["residencia_provincia_nombre", "medianaEdadProvincia"]], how="left", on = "residencia_provincia_nombre")
  data["ratioEdadProvincia"] = data["edad"] / data["medianaEdadProvincia"]

  data["conSintomas"] = np.where(data.fecha_inicio_sintomas.isnull() == False,1,0)

  columns = ["id_evento_caso", "ratioEdadGenero", "ratioEdadProvincia", "extranjero", "confirmado", "descartado",
  "otros", "conSintomas" , "ratioDescartadoConfirmadoProvincia", "sexo_F",
  "sexo_M", "sexo_NR",  "decileEdad", "financiamiento_Privado", "financiamiento_Público", "fallecido"]

  return data[columns]

In [64]:
def levantoModelo(contentFolder):
   return pickle.load(open("{}/ligthGbmModel.sav".format(contentFolder), 'rb'))

In [65]:
data = pd.read_csv("{}/Covid19Validacion.zip".format(contentFolder), compression="zip")
data = preprocesamiento(data)
modelo = levantoModelo(contentFolder)

probabilidades = modelo.predict_proba(data.drop(columns = ["id_evento_caso", "fallecido"]))[:,-1]
data["probabilidades"] = probabilidades
data["prediccion"] = np.where(data.probabilidades >= 0.011478807020738037,1,0)

In [2]:
data = pd.read_csv("{}/Covid19Validacion.zip".format(contentFolder), compression="zip")
data = preprocesamiento(data)

NameError: ignored

In [1]:
import requests
json_object = {"features": list(data.drop(columns=["id_evento_caso", "fallecido"]).loc[1].values)}
a = requests.post("https://us-central1-motordecision.cloudfunctions.net/covid_model", json = json_object)

NameError: ignored

In [ ]:
a.text

In [ ]:
pip freeze